## Online Simulation-Based Digital Twin: Nibelungenbrücke

This notebook introduces an online simulation-based digital twin developed for the Nibelungenbrücke bridge. The orchestration system is designed to interact dynamically with the user by gathering key inputs; such as time, physical parameters, and spatial positions to perform real-time simulations of the bridge structure.

The simulations are powered by the FenicSXConcrete package and support both thermomechanical and structural deflection analyses. This framework enables physics-based modeling and continuous digital monitoring of the bridge's behavior.

Note: 

After confirming that you're operating within the correct Python environment, if the model still raises any library-related errors, activate the following command to ensure that all required packages are properly installed!

In [ ]:
#import sys
#!{sys.executable} -m pip install git+https://github.com/BAMresearch/FenicsXConcrete pint gmsh pytest jsonschema pandas pyproj tqdm pvlib

from nibelungenbruecke.scripts.digital_twin_orchestrator.interface_test_DT import DigitalTwinInterface

DigitalTwinInterface manages model selection, input generation, and execution of the digital twin simulation.
## DIGITAL TWIN INTERFACE SHOULD NOT BE HERE, IMPORT IT FROM CODE
Focus is on the "real" system, no need to generate random inputs or anything. The thermal model works well enough.

## Launch the Digital Twin Interface

The loop below will guide you step by step. You will be prompted to:
- Run a simulation by randomizing material parameters.
- Switch between available simulation models.
- Exit the interface at any time.


In [ ]:
# Create the interface and start interaction loop
path = 'use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'
path = '../../../use_cases/nibelungenbruecke_demonstrator_self_weight_fenicsxconcrete/input/settings/digital_twin_default_parameters.json'

dt = DigitalTwinInterface(path) # This can be substituted with the orchestrator instead, but this is easier for now

In case of wanting to use real data from the Nibelungenbrücke monitoring system, provide your key to MKP's API:

In [ ]:
key="your_api_key_here"  # Replace with your actual API key
dt.set_api_key(key)

### Definition of input parameters

Define here the parameters with which you want to generate the simulation. The options are:


In [ ]:
simulation_parameters = {
    'simulation_name': 'TestSimulation',
    'model': 'TransientThermal_1',
    'start_time': '2023-01-01T00:00:00',
    'end_time': '2023-01-02T00:00:00',
    'time_step': '10min',
    'virtual_sensor_positions': [
        {'x': 0.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor1'},
        {'x': 1.0, 'y': 0.0, 'z': 0.0, 'name': 'Sensor2'}
        # Note: the real sensor positions are added automatically by the interface, so you don't need to specify them here.
    ],
    'full_field_results': False, # Set to True if you want full field results, the simulation will take longer and the results will be larger.
    'uncertainty_quantification': False, # Set to True if you want uncertainty quantification, the simulation will take longer and the results will be larger.
}

Now we run the simulation:

In [ ]:
dt.load(simulation_parameters) # Here we first load and then run, so that we can check the inputs before running the simulation and throw an error if something is wrong.
results = dt.run() # The plotting should be separated from the run, so that we can run the simulation without plotting if we want to.

### Results

The result at the chosen sensors are plotted below. The results can be downloaded from the path written below.

In [ ]:
# Results is probably just a dictionary of values with the sensor names as keys and the values as lists of results at each time step.
# The method should just be going over the keys defined as virtual sensors and plotting the results at those positions
# Passing the parameters as a dictionary to the method is also acceptable if needed.
# The option "uncertainty_quantification" is used to determine if the results should be plotted with uncertainty quantification or not.

dt.plot_results_at_virtual_sensors(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])
# The results file should be saved in an accessible path at the root, with metadata about the simulation and the results.
print("PATH TO RESULTS:", dt.get_results_path())

We can plot aswell a comparison between model response and real sensors when available:

In [ ]:
dt.plot_results_at_real_sensors(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])  # Plot the results, this can be done after the run or separately if needed.

Execute the digital twin simulation using the loaded model and parameters. This will simulate the bridge behavior based on the selected physics model (e.g., thermal or displacement) and input configuration.

## Additional results (only if run)

### Full-field response

Here it is the 3D model full field response. The corresponding files can be downloaded from the given path

In [ ]:
# The 3D model results are generated by activating the "paraview" parameter in the simulation parameters, which will save the results as xdmf and h5 files.
# The visualization can be done using pyvista, which is a wrapper around VTK that allows for easy 3D visualization in Python.
# The results can be visualized using the plot_full_field_response method, which will load the results file and plot the full field response.
# If implementing this as a method does not work, we can just use the pyvista library directly to load the results file and plot the full field response.
if simulation_parameters['full_field_results']:
    print("Full field results are enabled, plotting the full field response.")
    dt.plot_full_field_response(results, uncertainty_quantification=simulation_parameters['uncertainty_quantification'])  # Plot the full field response, this should be done using pyvista and the saved results file.
    print("PATH TO FULL FIELD RESULTS:", dt.get_full_field_results_path())